In [1]:
import os
import torch

In [2]:
os.chdir("/home/shuaman/video_sm/video_summarization")

In [3]:
!pwd

/home/shuaman/video_sm/video_summarization


In [4]:
from src.models import MSVA
from src.utils import VSMDataset, parse_configuration
from src.utils.utils_model import *

In [5]:
msva = MSVA()

In [6]:
path_weights_summe = "/home/shuaman/video_sm/MSVA/model_weights/summe_random_non_overlap_0.5359.tar.pth"
path_weights_tvsum = "/home/shuaman/video_sm/MSVA/model_weights/tvsum_random_non_overlap_0.6271.tar.pth"

In [7]:
msva.load_state_dict(torch.load(path_weights_tvsum))

<All keys matched successfully>

In [8]:
dict_paths = {
        'path_tvsum':"/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_tvsum_google_pool5.h5",
        'path_summe':"/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_summe_google_pool5.h5",
        'path_ovp':"/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_ovp_google_pool5.h5",
        'path_youtube':"/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_youtube_google_pool5.h5",
        'path_cosum':"/data/shuaman/video_summarization/datasets/processed_datasets/dataset_cosum_processed.h5",
#         'path_tvsum':"/data/shuaman/video_summarization/datasets/processed_datasets/dataset_tvsum_processed.h5",
#         'path_summe':"/data/shuaman/video_summarization/datasets/processed_datasets/dataset_summe_processed.h5",
}

In [9]:
dict_use_feature = get_flags_features("googlenet", "i3d")

In [10]:
dict_use_feature

{'googlenet': True,
 'resnext': False,
 'inceptionv3': False,
 'i3d_rgb': True,
 'i3d_flow': True,
 'resnet3d': False}

In [11]:
params = {
        'batch_size': 1,
        'num_workers': 4
        }


In [12]:
path_split = "/home/shuaman/video_sm/video_summarization/splits/vasnet_splits/summe_splits.json"
splits = parse_configuration(path_split)
split = splits[0]

In [13]:
split.keys()

dict_keys(['train_keys', 'test_keys'])

In [14]:
dataset_paths = get_paths('summe', 'canonical', **dict_paths)
dataset_paths

'/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_summe_google_pool5.h5'

In [15]:
training_generator, test_generator = get_dataloaders(dataset_paths, split, 
                                                     dict_use_feature, params)

In [16]:
it = 0
for i in training_generator:
    it += 1
    continue
print(it)

20


In [17]:
it = 0
for i in test_generator:
    it += 1
    continue
print(it)

5


In [18]:
device = torch.device("cuda:3")